In [ ]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add(url = "https://github.com/SimonEnsemble/Xtals.jl.git")

In [11]:
using DataFrames
using CSV
using Printf
using Statistics
using LinearAlgebra

In [12]:
using Xtals
using LightGraphs
using GraphPlot

In [13]:
mol_file = "0.mol"

"0.mol"

In [14]:
atoms, bonds, bondtypes = read_mol(joinpath(homedir(), "Documents", "GitHub", "MolecularSynthesis", "examples", mol_file))
#atoms = read_xyz(joinpath(homedir(), "Documents", "Grad School", "Research", "examples", "1.xyz"))

(Atoms{Cart}(47, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :H, :H, :H, :H, :O, :O, :O, :H], Cart([1.512 0.769 … 0.689 -0.316; 0.498 1.618 … 18.076 19.979; -0.014 -0.112 … 1.186 0.0])), {47, 49} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0), [1, 1, 2, 1, 5, 5, 5, 1, 1, 5  …  1, 1, 1, 1, 1, 2, 1, 1, 1, 1])

In [15]:
r = maximum([distance(atoms, i, j) for i = 1:atoms.n, j = 1:atoms.n])
box_length = 3*r

59.292220779795386

In [16]:
box = Box(box_length,box_length,box_length)
atoms=Frac(atoms, box)

Atoms{Frac}(47, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :H, :H, :H, :H, :O, :O, :O, :H], Frac([0.025500815791929896 0.012969660941795032 … 0.011620411428994488 -0.005329535575562089; 0.00839907821718326 0.02728857139639059 … 0.30486292741727833 0.33695820020302075; -0.00023611866474009164 -0.0018889493179207332 … 0.02000262402726776 0.0]))

In [17]:
crystal = Crystal("crystal.cif", box, atoms, Charges{Frac}(0), bonds, Xtals.SymmetryInfo())

Name: crystal.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 59.292221 Å. b = 59.292221 Å, c = 59.292221 Å
	Volume of unit cell: 208445.801115 Å³

	# atoms = 47
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 20,:O => 5,:C => 20)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [18]:
crystal2 = Crystal("tobacco_crystal.cif", box, atoms, Charges{Frac}(0), bonds, Xtals.SymmetryInfo())

Name: tobacco_crystal.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 59.292221 Å. b = 59.292221 Å, c = 59.292221 Å
	Volume of unit cell: 208445.801115 Å³

	# atoms = 47
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 20,:O => 5,:C => 20)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [14]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to X
Edge 4 => 6
atom X is connected to C
Edge 4 => 10
atom X is connected to C
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to H
Edge 7 => 8
atom C is connected to C
Edge 7 => 12
atom C is connected to H
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 44
atom C is connected to O
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 11 => 20
atom C is connected to C
Edge 15 => 41
atom C is connected to H
Edge 15 => 42
atom C is connected to H
Edge 15 => 43
atom C is connected to H
Edge 15 => 44
atom C is connected to O
Edge 16 => 17
atom C is connected to C
Edge 16 => 23
atom C is connected to H
Edge 17 => 18
atom C is connected to C
Edge 17 => 22
atom C is connected to H
Edge 18 => 19
atom C is connected to C
Edge 18 

In [19]:
### Function to identify carboxyl. When species is carbon, checks if it has two oxygen neighbors. 
### If it does, returns oxygen id's, anchor id, and oxygen's hydrogen id

function identify_carboxyl(crystal::Crystal, a::Int64)
   
    species = crystal.atoms.species[a]    #Identify species of selected atom
    nbs = neighbors(crystal.bonds, a)     #Identify neighbors of atom
    
    if species == :C                      #Identifying carboxylate starts with carbon
        oxygen_counter = 0
        oxygen_id = Int64[]
        X_id = 0
        H_id = 0
        for nb in nbs                     #Iterating through neighbors via identifying number
            species_nb = crystal.atoms.species[nb] #Get species of neighbor
            if species_nb == :O
                
                next_nbs = neighbors(crystal.bonds, nb)  #Get neighbors of oxygen (finding hydrogen)
                
                for next_nb in next_nbs
                    species_next_nb = crystal.atoms.species[next_nb]  #Get species of oxygen's neighbors
                    if species_next_nb == :H     #Identify hydrogen neighbor
                        H_id = next_nb
                    else
                    end
                end
                
                oxygen_counter += 1
                push!(oxygen_id, nb)         #Enter oxygen id into array
            else
                X_id = nb            #If neighbor of carboxylate carbon isn't oxygen, must be anchor
            end
        end
        
        if oxygen_counter == 2   #Assuming carboxylate carbon is only carbon with two oxygens bonded
        
            return true, oxygen_id, X_id, H_id
        else 
            return false, [], 0, 0
        end
    else
        return false, [], 0, 0
    end

end

identify_carboxyl (generic function with 1 method)

In [ ]:
identify_carboxyl(crystal2, 26)

In [20]:
#

keep = [true for i = 1:crystal2.atoms.n]

X_species = Symbol[]
X_ids = Int64[]

for a = 1:crystal2.atoms.n
       
   

    is_carboxyl, oxygen_ids, X_id, H_id = identify_carboxyl(crystal2, a)
    
    if is_carboxyl == true 
    
        push!(X_species, crystal2.atoms.species[X_id])
        push!(X_ids, X_id)
        crystal.atoms.species[X_id] = :X
        crystal2.atoms.species[X_id] = :X
        @assert length(oxygen_ids) == 2
        keep[oxygen_ids] .= false
        keep[a] = false
        keep[H_id] = false
        
    end


end

In [21]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal2.atoms.species[i]
    species_j = crystal2.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to X
Edge 4 => 6
atom X is connected to C
Edge 4 => 10
atom X is connected to C
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to H
Edge 7 => 8
atom C is connected to C
Edge 7 => 12
atom C is connected to H
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 44
atom C is connected to O
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 11 => 20
atom C is connected to C
Edge 15 => 41
atom C is connected to H
Edge 15 => 42
atom C is connected to H
Edge 15 => 43
atom C is connected to H
Edge 15 => 44
atom C is connected to O
Edge 16 => 17
atom C is connected to C
Edge 16 => 23
atom C is connected to H
Edge 17 => 18
atom C is connected to C
Edge 17 => 22
atom C is connected to H
Edge 18 => 19
atom C is connected to C
Edge 18 

In [22]:
tobacco_crystal = crystal2[BitArray(keep)]

Name: tobacco_crystal.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 59.292221 Å. b = 59.292221 Å, c = 59.292221 Å
	Volume of unit cell: 208445.801115 Å³

	# atoms = 39
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 18,:X => 2,:O => 1,:C => 16)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [26]:
#infer_bonds!(crystal, false)
infer_bonds!(tobacco_crystal, false)

┌ Warning: atom 1 = X in tobacco_crystal.cif is not bonded to any other atom.
└ @ Xtals C:\Users\kgeri\.julia\packages\Xtals\Hr2Rt\src\bonds.jl:430


false

In [27]:
write_xyz(tobacco_crystal)
write_xyz(crystal)

In [28]:
write_bond_information(crystal)
write_bond_information(tobacco_crystal)

Saving bond information for crystal crystal.cif to C:\Users\kgeri\Documents\GitHub\MolecularSynthesis\crystal_bonds.vtk.
Saving bond information for crystal tobacco_crystal.cif to C:\Users\kgeri\Documents\GitHub\MolecularSynthesis\tobacco_crystal_bonds.vtk.


In [15]:
write_mol2(crystal)
write_mol2(tobacco_crystal)

In [ ]:
write_cif(crystal, "crystal.cif")

Name: tobacco_crystal.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 59.292221 Å. b = 59.292221 Å, c = 59.292221 Å
	Volume of unit cell: 208445.801115 Å³

	# atoms = 39
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 18,:X => 2,:O => 1,:C => 16)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [22]:
for ed in edges(tobacco_crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = tobacco_crystal.atoms.species[i]
    species_j = tobacco_crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)


end

In [23]:
function center_mass(crystal::Crystal)
    xf_cm = sum(crystal.atoms.coords.xf, dims = 2)
    frctn_xf_cm = xf_cm ./ crystal.atoms.n
    new_coords = crystal.atoms.coords.xf .- frctn_xf_cm
    crystal.atoms.coords.xf .= mod.(crystal.atoms.coords.xf, 1)
    crystal.atoms.coords.xf .= new_coords
    
    return crystal
end

center_mass (generic function with 1 method)

In [24]:
tobacco_crystal = center_mass(tobacco_crystal)

Name: tobacco_crystal.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 59.292221 Å. b = 59.292221 Å, c = 59.292221 Å
	Volume of unit cell: 208445.801115 Å³

	# atoms = 39
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 18,:X => 2,:O => 1,:C => 16)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [25]:
write_mol2(tobacco_crystal)

In [ ]:
for edge in collect(edges(tobacco_crystal.bonds))
            dxf = tobacco_crystal.atoms.coords.xf[:, edge.src] - tobacco_crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf("%s\t%s\t%0.5f\t%s\n", tobacco_crystal.atoms.species[edge.src], tobacco_crystal.atoms.species[edge.dst],
                    norm(dxf), ". A")
end

In [ ]:

for ed in edges(crystal.bonds)
   
    #println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    dxf = crystal.atoms.coords.xf[:, i] - crystal.atoms.coords.xf[:, j]
    nearest_image!(dxf)
    norm_dxf = norm(dxf)
    println(norm_dxf)
    println("atom ", species_i, " is connected to ", species_j)

end

In [27]:
"""
    write_cif(crystal, filename; fractional_coords=true, number_atoms=true)
Write a `crystal::Crystal` to a .cif file with `filename::AbstractString`. If `filename` does
not include the .cif extension, it will automatically be added. the `fractional_coords` flag
allows us to write either fractional or Cartesian coordinates.
"""
function write_cif_Kai(crystal::Crystal, filename::AbstractString, X_species::Array{Symbol,1}, X_ids::Array{Int64,1};
		 fractional_coords::Bool=true, number_atoms::Bool=true)
    if has_charges(crystal)
        if crystal.atoms.n != crystal.charges.n
            error("write_cif assumes equal numbers of Charges and Atoms (or zero charges)")
        end
        if ! isapprox(crystal.charges.coords, crystal.atoms.coords)
            error("write_cif needs coords of atoms and charges to correspond.")
        end
    end



    # append ".cif" to filename if it doesn't already have the extension
    if ! occursin(".cif", filename)
        filename *= ".cif"
    end
    cif_file = open(filename, "w")
    # first line should be data_xtalname_PM
    if crystal.name == ""
        @printf(cif_file, "data_PM\n")
    else
        # don't include file extension!
        @printf(cif_file, "data_%s_PM\n", split(crystal.name, ".")[1])
    end

    @printf(cif_file, "_symmetry_space_group_name_H-M\t'%s'\n", crystal.symmetry.space_group)

    @printf(cif_file, "_cell_length_a\t%f\n", crystal.box.a)
    @printf(cif_file, "_cell_length_b\t%f\n", crystal.box.b)
    @printf(cif_file, "_cell_length_c\t%f\n", crystal.box.c)

    @printf(cif_file, "_cell_angle_alpha\t%f\n", crystal.box.α * 180.0 / pi)
    @printf(cif_file, "_cell_angle_beta\t%f\n", crystal.box.β * 180.0 / pi)
    @printf(cif_file, "_cell_angle_gamma\t%f\n", crystal.box.γ * 180.0 / pi)

    @printf(cif_file, "_symmetry_Int_Tables_number 1\n\n")
    @printf(cif_file, "loop_\n_symmetry_equiv_pos_as_xyz\n")
    for i in 1:size(crystal.symmetry.operations, 2)
        @printf(cif_file, "'%s,%s,%s'\n", crystal.symmetry.operations[:, i]...)
    end
    @printf(cif_file, "\n")

    @printf(cif_file, "loop_\n_atom_site_label\n_atom_site_type_symbol\n")
    if fractional_coords
        @printf(cif_file, "_atom_site_fract_x\n_atom_site_fract_y\n_atom_site_fract_z\n")
    else
        @printf(cif_file, "_atom_site_Cartn_x\n_atom_site_Cartn_y\n_atom_site_Cartn_z\n")
    end
    high_precision_charges = false # if, for neutrality, need high-precision charges
    if has_charges(crystal)
        @printf(cif_file, "_atom_site_charge\n")
        # if crystal will not be charge neutral to a 1e-5 tolerance when loading it
        #    into PorousMaterials.jl, then write higher-precision charges
        if abs(sum(round.(crystal.charges.q, digits=6))) > NET_CHARGE_TOL
            @info "writing high-precision charges for " * filename * ".\n"
            high_precision_charges = true
        end
    end

    
    nb_of_X = 0
    idx_to_label = ["" for i = 1:crystal.atoms.n]
    for i = 1:crystal.atoms.n
        # print label and type symbol
        
        if crystal.atoms.species[i] == :X
            nb_of_X += 1

            element = X_species[nb_of_X]
            
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                string(i),
                element)
        else
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                string(i),
                crystal.atoms.species[i])
        end
        
        # store label for this atom idx
        idx_to_label[i] = string(crystal.atoms.species[i]) *
                    string(i)
        # increment label
        
        if fractional_coords
            @printf(cif_file, "%f\t%f\t%f", crystal.atoms.coords.xf[:, i]...)
        else
            @printf(cif_file, "%f\t%f\t%f", (crystal.box.f_to_c * crystal.atoms.coords.xf[:, i])...)
        end
        if has_charges(crystal)
            if high_precision_charges
                @printf(cif_file, "\t%.10f\n", crystal.charges.q[i])
            else
                @printf(cif_file, "\t%f\n", crystal.charges.q[i])
            end
        else
            @printf(cif_file, "\n")
        end
    end

    # only print bond information if it is in the crystal
    if ne(crystal.bonds) > 0
        if ! number_atoms
             error("must label atoms with numbers to write bond information.\n")
        end
        # print column names for bond information
        @printf(cif_file, "\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_ccdc_geom_bond_type\n")

        for edge in collect(edges(crystal.bonds))
            dxf = crystal.atoms.coords.xf[:, edge.src] - crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf(cif_file, "%s\t%s\t%0.5f\t%s\n", idx_to_label[edge.src], idx_to_label[edge.dst],
                    norm(dxf), ". A")
        end
    end
    close(cif_file)
end

write_cif_Kai

In [ ]:
X_species

In [ ]:
X_ids

In [28]:
infer_bonds!(tobacco_crystal, false)

┌ Warning: atom 1 = X in tobacco_crystal.cif is not bonded to any other atom.
└ @ Xtals C:\Users\kgeri\.julia\packages\Xtals\Hr2Rt\src\bonds.jl:430


false

In [29]:
write_cif_Kai(tobacco_crystal, "tobacco_crystal.cif", X_species, X_ids)

In [ ]:
mol_file = replace(mol_file, "mol" => "cif")
test_file_name = string("Test", mol_file)

In [ ]:
write_cif_Kai(tobacco_crystal, test_file_name, X_species, X_ids)

In [ ]:
crystal = tobacco_crystal
for edge in collect(edges(crystal.bonds))
            dxf = crystal.atoms.coords.xf[:, edge.src] - crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf("%s\t%s\t%0.5f\t%s\n", crystal.atoms.species[edge.src], crystal.atoms.species[edge.dst],
                    norm(dxf), ". A")
end